In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

In [2]:
# Preprocessing for the machine learning data

from pathlib import Path
import requests

def download_file(year: int , month: int) -> Path:
    """"""
    URL = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
    response = requests.get(URL)

    if response.status_code == 200:
        path = f"../data/raw/rides_{year}-{month:02d}.parquet"
        open(path, 'wb').write(response.content)
        return path
    else:
        raise Exception(f"Failed to download {URL}")

In [3]:
download_file(2022,1)

'../data/raw/rides_2022-01.parquet'

In [3]:
rides = pd.read_parquet("../data/raw/rides_2022-01.parquet")

# rides.head(25) show the entire rides dataframe

In [4]:
rides = rides[["tpep_pickup_datetime","PULocationID"]]

rides.rename(columns = {
    "tpep_pickup_datetime":"pickup_time",
    "PULocationID": "pickup_location"
},inplace=True)

rides.head()

,pickup_time,pickup_location
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [5]:
# Data validation and data cleaning:
## test for inaccuracies in the data labelling of the dates

rides['pickup_time'].describe()

/var/folders/r9/9kmctj5907z_qfw6hp7s7bb40000gn/T/ipykernel_82144/3383500000.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_time'].describe()


count                 2463931
unique                1423522
top       2022-01-26 07:57:00
freq                       12
first     2008-12-31 22:23:09
last      2022-05-18 20:41:57
Name: pickup_time, dtype: object

In [6]:
rides = rides[rides.pickup_time >= '2022-01-01']
rides = rides[rides.pickup_time < '2022-02-01']

rides['pickup_time'].describe()

/var/folders/r9/9kmctj5907z_qfw6hp7s7bb40000gn/T/ipykernel_82144/2547043621.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_time'].describe()


count                 2463879
unique                1423471
top       2022-01-26 07:57:00
freq                       12
first     2022-01-01 00:00:08
last      2022-01-31 23:59:58
Name: pickup_time, dtype: object

In [14]:
## read the data to a new file of transformed data

rides.to_parquet('../data/cleaned/cleaned_rides_2022-01.parquet')